In [1]:
import tensorflow as tf

In [2]:
from read_data import dataset, dataset2

In [3]:
import os

In [4]:
import numpy as np

In [5]:
import random

In [7]:
#定义存储地址与名称
CNN = '/home/aa/Vehicle_License_Plate_Recognition/data/cnn'
cnn = 'License_plate'
#定义模型文件所在的文件夹，若不存在则自动创建
ckpt = tf.train.latest_checkpoint(CNN)
if not ckpt:
    if not os.path.exists(CNN):
        os.mkdir(CNN)

In [8]:
# 卷积神经网络（有正则化的添加与使用）
def weight_variable(shape):  
    # 使用截断正态分布生成卷积核
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  
  
def bias_variable(shape):  
    # 使用relu激活函数，用一个正偏置值较准
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  
  
def conv2d(x, W):  
    #定义卷积层
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  
def max_pool_2x2(x):  
    # 定义池化层
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  


# min_next_batch_tfr(随机批次载入数据)
def min_next_batch_tfr(image, label, num=50, num1=500): 
    images = np.zeros((num, 1152))
    labels = np.zeros((num, 34))
    for i in range(num):
        temp = random.randint(0, num1-1)
        images[i, :] = image[temp]
        labels[i, :] = label[temp]

    return images, labels


x = tf.placeholder(tf.float32, [None, 1152])
y_ = tf.placeholder(tf.float32, [None, 34])
keep_prob = tf.placeholder("float")  

# 第一层 先卷积再池化，卷积核尺寸为3*3, 通道数为1，输出通道为32，池化输出后应为24*12*32
W_conv1 = weight_variable([3, 3, 1, 32])  
b_conv1 = bias_variable([32])  
# 格式转换
x_image = tf.reshape(x, [-1, 48, 24, 1])  
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  

# 第二层 先卷积再池化，卷积核尺寸为3*3, 输入通道为32，输出通道为64，池化输出后应为12*6*64
W_conv2 = weight_variable([3, 3, 32, 64])  
b_conv2 = bias_variable([64])  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
  
# 第三层 先卷积再池化，卷积核尺寸为3*3, 输入通道为64，输出通道为96，池化输出后应为6*3*96
W_conv3 = weight_variable([3, 3, 64, 96])  
b_conv3 = bias_variable([96])  
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)  
h_pool3 = max_pool_2x2(h_conv3)  

# 此时图片格式简化为6*3，进入第四层全连接层,输入维数6*3*96, 输出维数为1024
W_fc1 = weight_variable([6 * 3 * 96, 1024])  
b_fc1 = bias_variable([1024])  
h_pool2_flat = tf.reshape(h_pool3, [-1, 6*3*96])  
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  
# 使用drop out防止过拟合（正则化）
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  

# 第五层，输入1024维，输出34维，即0~33分类
W_fc2 = weight_variable([1024, 34])  
b_fc2 = bias_variable([34])   
y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)  
saver=tf.train.Saver(max_to_keep=1)

# 损失函数，交叉熵 
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))  
# 使用adam优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) 
# 准确度计算
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))  
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  

In [9]:
x_train, y_train = dataset2('train', 5168)
x_test, y_test = dataset2('test', 1734)

training...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
done
training...
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
done


In [10]:
# 训练模型，训练过程中每一百次插播验证集当前识别率
with tf.Session() as sess: 
    # 运行会话
    sess.run(tf.global_variables_initializer())
    # 变量定义
    step = 0
    j = 0 # 有三代loss都比minloss大则保存模型
    losslist = []
    minloss = 100000
    # 若模型存在，自动加载模型进会话
    ckpt = tf.train.latest_checkpoint(CNN)
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(CNN, cnn)) + 1:])
    ckptname=os.path.join(CNN, cnn)
    
    # 开启线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 训练
    for i in range(4000):
        batch = min_next_batch_tfr(x_train, y_train, 50, 5168)
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
        # 每一百次用验证集测试一次
        if i % 100 == 0:
            train_accuracy, loss = sess.run([accuracy, cross_entropy], feed_dict={
                x: x_test, y_: y_test, keep_prob: 1})
            print("step %d, validating accuracy %g, loss is %g" % (i, train_accuracy, loss))
            losslist.append(loss)
        # 保存损失最低的模型
        if minloss > loss:
            minloss = loss
            saver.save(sess,ckptname,global_step=i)
        if losslist[-1] > minloss and losslist[-2] > minloss and losslist[-3] > minloss:
            break
    
    # 损失函数列表
    print(losslist)
    print(minloss)
    coord.request_stop() 
    coord.join(threads)

step 0, validating accuracy 0.0334487, loss is 12608.7
step 100, validating accuracy 0.328143, loss is 5025.43
step 200, validating accuracy 0.696655, loss is 3540.35
step 300, validating accuracy 0.859285, loss is 2186.88
step 400, validating accuracy 0.89504, loss is 1385.99
step 500, validating accuracy 0.915802, loss is 968.809
step 600, validating accuracy 0.9406, loss is 686.146
step 700, validating accuracy 0.94579, loss is 545.651
step 800, validating accuracy 0.963668, loss is 428.074
step 900, validating accuracy 0.970588, loss is 330.298
step 1000, validating accuracy 0.975779, loss is 292.882
step 1100, validating accuracy 0.978662, loss is 247.698
step 1200, validating accuracy 0.980969, loss is 225.71
step 1300, validating accuracy 0.982699, loss is 208.798
step 1400, validating accuracy 0.983276, loss is 181.239
step 1500, validating accuracy 0.986159, loss is 169.094
step 1600, validating accuracy 0.987313, loss is 155.553
step 1700, validating accuracy 0.989619, loss i

In [11]:
# 验证数据集判断模型效果
with tf.Session() as sess: 
    # 运行会话
    sess.run(tf.global_variables_initializer())
    step=0
    # 若模型存在，自动加载模型进会话
    ckpt = tf.train.latest_checkpoint(CNN)
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(CNN, cnn)) + 1:])
            
    # 开启线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 测试
    print("test accuracy %g" % accuracy.eval(feed_dict={
        x: x_test, y_: y_test, keep_prob: 1.0}))
    L_lmax = {}.fromkeys(range(34), 0)
    L_lsame = {}.fromkeys(range(34), 0)
    CL = {0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',
     10:'A',11:'B',12:'C',13:'D',14:'E',15:'F',16:'G',17:'H',18:'J',19:'K',
     20:'L',21:'M',22:'N',23:'P',24:'Q',25:'R',26:'S',27:'T',28:'U',29:'V',
     30:'W',31:'X',32:'Y',33:'Z'}
    prediction_label = tf.argmax(y_conv.eval(feed_dict={
        x: x_test, keep_prob: 1.0}), 1)
    for i in range(len(x_test)):
        if i%100 ==0:
            print(i)
        L_lmax[int(np.argmax(y_test[i]))] += 1
        if np.argmax(y_test[i]) == prediction_label[i].eval():
            L_lsame[int(np.argmax(y_test[i]))] += 1
    for i in range(34):
        recall_rate = L_lsame[i]/L_lmax[i]
        print("标签 %s 的召回率为 %f" %(CL[i],recall_rate))
        
    coord.request_stop() 
    coord.join(threads)

INFO:tensorflow:Restoring parameters from /home/aa/Vehicle_License_Plate_Recognition/data/cnn/License_plate-2700
test accuracy 0.990773
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
标签 0 的召回率为 0.980392
标签 1 的召回率为 1.000000
标签 2 的召回率为 0.960784
标签 3 的召回率为 1.000000
标签 4 的召回率为 0.980392
标签 5 的召回率为 1.000000
标签 6 的召回率为 0.980392
标签 7 的召回率为 1.000000
标签 8 的召回率为 0.980392
标签 9 的召回率为 1.000000
标签 A 的召回率为 1.000000
标签 B 的召回率为 1.000000
标签 C 的召回率为 1.000000
标签 D 的召回率为 0.960784
标签 E 的召回率为 1.000000
标签 F 的召回率为 0.980392
标签 G 的召回率为 0.980392
标签 H 的召回率为 1.000000
标签 J 的召回率为 0.980392
标签 K 的召回率为 1.000000
标签 L 的召回率为 1.000000
标签 M 的召回率为 0.980392
标签 N 的召回率为 1.000000
标签 P 的召回率为 1.000000
标签 Q 的召回率为 0.980392
标签 R 的召回率为 1.000000
标签 S 的召回率为 0.960784
标签 T 的召回率为 0.980392
标签 U 的召回率为 1.000000
标签 V 的召回率为 1.000000
标签 W 的召回率为 1.000000
标签 X 的召回率为 1.000000
标签 Y 的召回率为 1.000000
标签 Z 的召回率为 1.000000
